In [10]:
from waveorder.io.writer import WaveorderWriter
from infectedcell_annotator.functions import gather_image_information_opencell, gather_image_information, load_image_stack
import pandas as pd
import os
import tifffile as tiff

# InfectedCell Datasets

### Convert raw data to ome-zarr

In [6]:
raw_data_dir = '/gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/raw_data'
save_dir = '/gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/data_convert_test/zarrs'
plate_map_path = '/gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/TICM0009-1 Platemap.csv'
opencell = True # set to false if this is timelapse data acquired with MDA

plate_map = pd.read_csv(plate_map_path)
writer = WaveorderWriter(save_dir)

In [7]:
if opencell:
    gfp_idx = 1
    dapi_idx = 0
    plate_map = pd.read_csv(plate_map_path)
    experiment_dict = gather_image_information_opencell(raw_data_dir, plate_map)
    labels = list(experiment_dict.keys())
else:
    experiment_dict = gather_image_information(raw_data_dir)
    labels = list(experiment_dict.keys())

    sub_dict = experiment_dict[labels[0]]
    fp = sub_dict[list(sub_dict.keys())[0]]['FOVs'][0]['file_path']

    meta = tiff.TiffFile(fp).micromanager_metadata
    for idx, chan in enumerate(meta['Summary']['ChNames']):
        if 'GFP' in chan:
            gfp_idx = idx
        elif 'DAPI' in chan:
            dapi_idx = idx
        else:
            continue

for label in labels:
    
    writer.create_zarr_root(label+'_Mock.zarr')
    
    for fov_mock in experiment_dict[label]['Mock']['FOVs']:
        
        stack = load_image_stack(fov_mock['file_path'], gfp_idx, dapi_idx, opencell)
        
        data_shape = (1, 2, stack.shape[-3], stack.shape[-2], stack.shape[-1])
        chunk_size = (1, 1, 1, stack.shape[-2], stack.shape[-1])
        chan_names = ['GFP', 'DAPI']
        dtype = stack.dtype

        name = f'{label}_Mock_FOV{fov_mock["FOV"]}'
        
        writer.init_array(position=fov_mock["FOV"],
                         data_shape=data_shape,
                         chunk_size=chunk_size,
                         chan_names=chan_names,
                         position_name=name)
        
        writer.write(stack, p=fov_mock["FOV"], t=0)

    
    writer.create_zarr_root(label+'_Infected.zarr')

    for fov_inf in experiment_dict[label]['Infected']['FOVs']:
        
        stack = load_image_stack(fov_mock['file_path'], gfp_idx, dapi_idx, opencell)
        
        data_shape = (1, 2, stack.shape[-3], stack.shape[-2], stack.shape[-1])
        chunk_size = (1, 1, 1, stack.shape[-2], stack.shape[-1])
        chan_names = ['GFP', 'DAPI']
        dtype = stack.dtype

        name = f'{label}_Infected_FOV{fov_inf["FOV"]}'
        
        writer.init_array(position=fov_inf["FOV"],
                         data_shape=data_shape,
                         chunk_size=chunk_size,
                         chan_names=chan_names,
                         position_name=name)
        
        writer.write(stack, p=fov_inf["FOV"], t=0)

Creating new zarr store at /gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/data_convert_test/zarrs/RAC1_Mock.zarr
Creating new zarr store at /gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/data_convert_test/zarrs/RAC1_Infected.zarr
Creating new zarr store at /gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/data_convert_test/zarrs/TOMM70_Mock.zarr
Creating new zarr store at /gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/data_convert_test/zarrs/TOMM70_Infected.zarr
Creating new zarr store at /gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/data_convert_test/zarrs/NPM1_Mock.zarr
Creating new zarr store at /gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/data_convert_test/zarrs/NPM1_Infected.zarr
Creating new zarr store at /gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/data_convert_test/zarrs/KRAS_Mock.zarr
Creating new zarr store at /gpfs/gpfsML/ML_group/opencell-infection-microscopy/TI

### Convert raw data to tiff stacks

In [8]:
raw_data_dir = '/gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/raw_data'
save_dir = '/gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/data_convert_test/Tiffs'
plate_map_path = '/gpfs/gpfsML/ML_group/opencell-infection-microscopy/TICM0009-1/TICM0009-1 Platemap.csv'
opencell = True

plate_map = pd.read_csv(plate_map_path)

In [12]:
if opencell:
    gfp_idx = 1
    dapi_idx = 0
    plate_map = pd.read_csv(plate_map_path)
    experiment_dict = gather_image_information_opencell(raw_data_dir, plate_map)
    labels = list(experiment_dict.keys())
else:
    experiment_dict = gather_image_information(raw_data_dir)
    labels = list(experiment_dict.keys())

    sub_dict = experiment_dict[labels[0]]
    fp = sub_dict[list(sub_dict.keys())[0]]['FOVs'][0]['file_path']

    meta = tiff.TiffFile(fp).micromanager_metadata
    for idx, chan in enumerate(meta['Summary']['ChNames']):
        if 'GFP' in chan:
            gfp_idx = idx
        elif 'DAPI' in chan:
            dapi_idx = idx
        else:
            continue

for label in labels:
    
    print('Saving label', label)
        
    for fov_mock in experiment_dict[label]['Mock']['FOVs']:
        
        stack = load_image_stack(fov_mock['file_path'], gfp_idx, dapi_idx, opencell)

        name = f'{label}_Mock_FOV{fov_mock["FOV"]}.tif'
        
        tiff.imsave(os.path.join(save_dir, name), stack)
    
    for fov_inf in experiment_dict[label]['Infected']['FOVs']:
        
        stack = load_image_stack(fov_mock['file_path'], gfp_idx, dapi_idx, opencell)

        name = f'{label}_Infected_FOV{fov_inf["FOV"]}.tif'
        
        tiff.imsave(os.path.join(save_dir, name), stack)

Saving label RAC1
Saving label TOMM70
Saving label NPM1
Saving label KRAS
Saving label EFM
Saving label LAMP1
Saving label PXN
Saving label MAP1LC3B
Saving label POLR1A
Saving label LMAN1
Saving label GPR107
Saving label MAP4
Saving label GOLGA2
Saving label HISTH12BE
Saving label G3BP1
Saving label RTN4
Saving label CEP131
Saving label PEX3
Saving label LMNA
Saving label SEC61B
Saving label DCB1B
